# Using Spark/`{sparklyr}` to extract datasets

Run this notebook with a Spark JupyterLab Cluster with at least:

- 4 nodes (`mem1_ssd1_v2_v4`)
- `Hail` configuration

If our query is takes longer than 2 minutes to execute, `dx extract_dataset` will timeout. In that case, we will start up a Spark instance of JupyterLab and run the SparkSQL directly in the notebook.

1. Extract the relevant dictionary information
2. Connect to our Spark Cluster using `sparklyr`
3. Extract the dataset SQL using the `--sql` option with `dx extract_dataset` and save output to a file
4. Load query from file, clean the SQL
5. Use `dbGetQuery()` to load the dataset as a `data.frame`
6. Use `readr::write_csv()` to save `data.frame` as CSV file
7. Upload our CSV file back to project storage using `dx upload`.

Optionally, we can:

1. Make our Query a temporary table in Spark
2. Filter our table in Spark
3. Utilize Spark methods (Machine Learning) on our Spark table
4. Export our Spark table for further work

In [ ]:
#install.packages("sparklyr")


## Generate SQL from `dx extract_dataset`

If we use the `--sql` flag, the query will not execute, but the SparkSQL needed to execute the query will be saved. 

We'll use the `-o` option to save our query as `cohort.sql`.

In [ ]:
system("dx extract_dataset record-G5Ky4Gj08KQYQ4P810fJ8qPp -ddd")

Then we can pass in a list of entity/fields to `dx extract dataset` along with the `-sql` option to retrieve the SQL we need to retrieve our cohort. We'll use `glue::glue()` to build our `dx extract_dataset()` command and then execute our command in Bash using `system()`.

In [ ]:
field_list <- 'participant.eid,participant.p21022,participant.p1508_i0,participant.p1508_i1,participant.p1508_i2,participant.p41202,participant.p31,participant.p41226,participant.p3159_i0,participant.p3159_i1,participant.p3159_i2'

cohort <- "record-G5Ky4Gj08KQYQ4P810fJ8qPp"

cohort_template <- glue::glue("dx extract_dataset {cohort} --fields {field_list} --sql -o cohort.sql")

cohort_template

system(cohort_template)
list.files()

## Connect to Spark Cluster

The first thing we need to do is connect to the Spark Cluster. We'll use the `{sparklyr}` and `{DBI}` packages to connect.

Make sure to only connect once - Spark will throw an error if you try to connect twice by rerunning this code block.

To fix this, you will need to restart the kernel using **Kernel > Restart Kernel*

In [ ]:
library(sparklyr)
library(DBI)
port <- Sys.getenv("SPARK_MASTER_PORT")
master <- paste("spark://master:", port, sep = '')
sc = spark_connect(master)

## Load SQL from file

We'll read the `cohort.sql` file using `read_file()` and then execute the SQL using `dbGetQuery()`.

Here's what the actual query looks like:

In [ ]:
library(readr)
retrieve_sql <-read_file("cohort.sql")
retrieve_sql <- gsub("[;\n]", "", retrieve_sql)
retrieve_sql

## Execute Our Query with DBI/sparklyr

Now we can execute our SQL with `dbGetQuery()`. Note we utilize our Spark Connection (`sc`) to execute it.

Note that the `data.frame` returned by `dbGetQuery()` is not identical to the `.csv` file generated by `dx extract_dataset`. 

`dbGetQuery()` returns columns that have multiple values per lines as list columns. `{xvhelper}` can transform these columns into comma separated strings.

In [ ]:
df <- dbGetQuery(sc, retrieve_sql)
head(df)

In [ ]:
library(xvhelper)

    
data_dict <- readr::read_csv("female_coffee_3.0.data_dictionary.csv", show_col_types = FALSE)
coding_dict <- readr::read_csv("female_coffee_3.0.codings.csv", show_col_types=FALSE)

merged_dict <- xvhelper::merge_coding_data_dict(coding_dict, data_dict)

decoded <- df |>
 xvhelper::decode_df(merged_dict)


In [ ]:
head(decoded)

## Decode Column Names

We can also decode the column names:

In [ ]:
decoded_cols <- decoded |>
    xvhelper::decode_column_names(merged_dict)

head(decoded_cols)

## Working with Extracted Pheno Data in Spark

We can also load our data as a temporary table in Spark, which can be very useful when working with very large datasets.

This lets us leverage the power of Spark and the `{sparklyr}` package.

We'll first take our SQL query and use `CREATE TEMPORARY VIEW` to create a temporary table as a Spark DataFrame. Then we can do our Spark operations on this temporary table.

In [ ]:
temp_table_sql <- glue::glue("CREATE TEMPORARY VIEW pheno AS {retrieve_sql}")

result <- DBI::dbSendQuery(sc, temp_table_sql)

Now we can register the data frame (`pheno`) as a Spark dataframe using `dplyr::tbl()`. 

Once we do that, we're able to leverage `sparklyr` to do `dplyr` like filtering:

In [ ]:
new_df <- dplyr::tbl(sc, "pheno")

filtered_df <- new_df |> dplyr::filter(`participant.p21022` > 50)

In [ ]:
head(filtered_df)

In [ ]:
colnames(new_df)

In [ ]:
new_df |> 
    dplyr::mutate(`participant.p31` = as.character(`participant.p31`)) |>
    dplyr::count(`participant.p31`)


Once we're done with working with `filtered_df` in spark, we can pull it into memory using `collect()`. Then we can annotate the data as usual.

In [ ]:
in_memory <- collect(filtered_df)
head(in_memory)